<a href="https://colab.research.google.com/github/Psubrat98/ML-Projects-With-Python/blob/main/Deep%20Learning/MY%20Project/Solar%20PV%20Panel/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading Dependent Libraries

In [1]:
from __future__ import division, print_function
# coding=utf-8
import sys
import os
import glob
import re
import numpy as np
import tensorflow as tf
import tensorflow as tf

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [2]:
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Keras

In [3]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

Flask utils

In [4]:
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename

Define a flask app

In [5]:
app = Flask(__name__)

Model saved with Keras model.save()

In [8]:
MODEL_PATH ='/content/model_Solar_PV_resnet50.h5'


Loading Model

In [10]:
model = load_model(MODEL_PATH)

Creating a Function for Model Prediction

In [12]:
def model_predict(img_path, model):
    print(img_path)
    img = image.load_img(img_path, target_size=(224, 224))

    # Preprocessing the image
    x = image.img_to_array(img)
    # x = np.true_divide(x, 255)
    ## Scaling
    x=x/255
    x = np.expand_dims(x, axis=0)
   

    # Be careful how your trained model deals with the input
    # otherwise, it won't make correct prediction!
   # x = preprocess_input(x)

    preds = model.predict(x)
    preds=np.argmax(preds, axis=1)
    if preds==0:
        preds="PV module with Dust drop particles"
    elif preds==1:
        preds="PV module with less Dust"
    else:
        preds="PV module with more Dust"

    return preds

In [13]:
@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')

In [15]:
@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']

        # Save the file to ./uploads
        basepath = os.path.dirname(__file__)
        file_path = os.path.join(
            basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)

        # Make prediction
        preds = model_predict(file_path, model)
        result=preds
        return result
    return None

In [ ]:
if __name__ == '__main__':
    app.run(port=5001,debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


INFO:werkzeug: * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
INFO:werkzeug: * Restarting with stat
